In [ ]:
import torch
import torch.nn as nn
import d2l.torch as d2l

## Incepton块

In [ ]:
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, c1, c2, c3, c4):
        super().__init__()
        self.path1 = nn.Sequential(
            nn.Conv2d(in_channels, c1, kernel_size=1),
            nn.ReLU()
        )
        self.path2 = nn.Sequential(
            nn.Conv2d(in_channels, c2[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.path3 = nn.Sequential(
            nn.Conv2d(in_channels, c3[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2),
            nn.ReLU()
        )
        self.path4 = nn.Sequential(
            nn.MaxPool2d(3, 1, 1),
            nn.Conv2d(in_channels, c4, 1),
            nn.ReLU()
        )

    def forward(self, X):
        Y_1 = self.path1(X)
        Y_2 = self.path2(X)
        Y_3 = self.path3(X)
        Y_4 = self.path4(X)
        Y = torch.cat((Y_1, Y_2, Y_3, Y_4), dim=1)
        return Y

In [1]:
class Inception(nn.Module):
    def __init__(self):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),  # [64, 112, 112]
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)  # [64, 56, 56]
        )
        self.block2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=1, stride=1),  # [64, 56, 56]
            nn.ReLU(),
            nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),  # [192, 56, 56]
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1)  # [192, 28, 28]
        )
        self.block3 = nn.Sequential(
            InceptionBlock(192, 64, (96, 128), (16, 32), 32),  # [256, 28, 28]
            InceptionBlock(256, 128, (128, 192), (32, 96), 64),  # [480, 28, 28]
            nn.MaxPool2d(3, 2, 1)  # [480, 14, 14]
        )
        self.block4 = nn.Sequential(
            InceptionBlock(480, 192, (96, 208), (16, 48), 64),  # [512, 14, 14]
            InceptionBlock(512, 160, (112, 224), (24, 64), 64),  # [512, 14, 14]
            InceptionBlock(512, 160, (112, 224), (24, 64), 64),  # [512, 14, 14]
            InceptionBlock(512, 160, (112, 224), (24, 64), 64),  # [512, 14, 14]
            InceptionBlock(528, 256, (160, 320), (32, 128), 128),  # [832, 14, 14]
            nn.MaxPool2d(3, 2, 1)  # [832, 7, 7]
        )
        self.block5 = nn.Sequential(
            InceptionBlock(832, 256, (160, 320), (32, 128), 128),  # [832, 7, 7]
            InceptionBlock(832, 384, (192, 384), (48, 128), 128),  # [1024, 7, 7]
            nn.AdaptiveAvgPool2d((1, 1))  # [1024, 1, 1]
        )
        self.fc = nn.Sequential(
            nn.Flatten(),  # [1024]
            nn.Linear(1024, 10)  #[10]
        )

    def forward(self, X):
        X = X.reshape(-1, 1, 224, 224)
        X = self.block1(X)
        X = self.block2(X)
        X = self.block3(X)
        X = self.block4(X)
        X = self.block5(X)
        y = self.fc(X)
        return y

SyntaxError: invalid syntax (1754348667.py, line 12)